In [ ]:
import sparql
from tabulate import tabulate

In [1]:
# set up end point and prefix
endpoint = "http://virtuoso.nkn.uidaho.edu:8890/sparql/"

# attach SPARQL querry prefix. Note: the graph for our study should be updated
sparql_prefix = """ 
                prefix dc: <http://purl.org/dc/elements/1.1/>
                prefix gts: <http://resource.geosciml.org/ontology/timescale/gts#>
                prefix skos: <http://www.w3.org/2004/02/skos/core#>
                prefix time: <http://www.w3.org/2006/time#>
                prefix ts: <http://resource.geosciml.org/vocabulary/timescale/> 
                prefix isc: <http://resource.geosciml.org/classifier/ics/ischart/> 
        """

In [2]:
sparql_prefix1 =sparql_prefix + """
                   SELECT DISTINCT ?schURI ?lbl
                   WHERE
                   {
                     GRAPH <http://deeptimekb.org/iscallnew> 
                     {
                        ?schURI a skos:ConceptScheme ;
                                rdfs:label ?lbl .
                        FILTER regex(?lbl, "International", "i")
                     }      
                   }
                   ORDER BY DESC (?schURI)
                   LIMIT 1
                  """

result = sparql.query(endpoint, sparql_prefix1)
rows = [sparql.unpack_row(row) for row in result]
rows=rows[0]
print(rows)

['http://resource.geosciml.org/vocabulary/timescale/isc2018-08', 'International Chronostratigraphic Chart (2018)']


In [3]:
sparql_prefix+=f"""
                   SELECT   DISTINCT ?begTimeValue ?endTimeValue
                   WHERE
                   {'{'}
                     GRAPH <http://deeptimekb.org/iscallnew>
                     {'{'}
                          {'{'} #for the pattern of local vocabulary
                                 ?tconcept  a gts:GeochronologicEra ;
                                                 rdfs:label ?label .
                                 FILTER (lang(?label) = "en")
                                 FILTER strstarts(?label, "Jurassic").
                                 ?tconcept time:hasBeginning ?beg ;
                                 time:hasEnd ?end .
                                 ?beg time:inTemporalPosition ?begTime .
                                 ?end time:inTemporalPosition ?endTime .
                                 ?begTime time:numericPosition ?begTimeValue .
                                 ?endTime time:numericPosition ?endTimeValue .
                          {'}'}
                                                 UNION
                          {'{'} #for the pattern of local vocabulary 
                             ?tconcept a gts:GeochronologicEra ;  
                                       rdfs:label ?label .
                             FILTER (lang(?label) = "en")
                             FILTER strstarts(?label, "Jurassic").
                             ?tconcept dc:description
                             [
                                time:hasBeginning ?beg ;
                                time:hasEnd ?end ;
                                skos:inScheme  <{rows[0]}>
                             ].
                             
                          ?beg time:inTemporalPosition ?begTime .
                          ?end time:inTemporalPosition ?endTime .
                          
                          ?begTime dc:description
                              [
                                 time:numericPosition ?begTimeValue ;
                                 skos:inScheme  <{rows[0]}>
                               ].
                          ?endTime dc:description
                              [
                                 time:numericPosition ?endTimeValue ;
                                 skos:inScheme  <{rows[0]}>
                              ] .
                             {'}'}
                     {'}'}
                   {'}'}
                  """

In [4]:
result = sparql.query(endpoint, sparql_prefix)
rows = [sparql.unpack_row(row) for row in result]
rows = [row + [abs(float(row[0])-float(row[1]))] for row in rows]
print(tabulate(rows, result.variables+['duration']))

  begTimeValue    endTimeValue    duration
--------------  --------------  ----------
         201.3             145        56.3
